In [1]:
!pip install opendatasets --quiet
import opendatasets as od
od.download('https://www.kaggle.com/datasets/emmarex/plantdisease')

Skipping, found downloaded files in ".\plantdisease" (use force=True to force download)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import torch
from torch import nn
from torch.optim import Adam, SGD
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
import os

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [17]:
root_path = 'plantdisease/PlantVillage/'
img_path = []
labels_path = []

for label in os.listdir(root_path):
    for item in os.listdir(f'{root_path}/{label}'):
        img_path.append(f'{root_path}/{label}/{item}')
        labels_path.append(label)
        
print(f'Number of Images: {len(img_path)}')

Number of Images: 20638


In [18]:
# Creation of our Dataframe
data_df = pd.DataFrame(zip(img_path, labels_path), columns = ['image_path', 'label'])

# Print the distribution of data among classes and the format of our DataFrame.
print(data_df['label'].value_counts())
data_df.head()

label
Tomato__Tomato_YellowLeaf__Curl_Virus          3208
Tomato_Bacterial_spot                          2127
Tomato_Late_blight                             1909
Tomato_Septoria_leaf_spot                      1771
Tomato_Spider_mites_Two_spotted_spider_mite    1676
Tomato_healthy                                 1591
Pepper__bell___healthy                         1478
Tomato__Target_Spot                            1404
Potato___Early_blight                          1000
Potato___Late_blight                           1000
Tomato_Early_blight                            1000
Pepper__bell___Bacterial_spot                   997
Tomato_Leaf_Mold                                952
Tomato__Tomato_mosaic_virus                     373
Potato___healthy                                152
Name: count, dtype: int64


,image_path,label
0,plantdisease/PlantVillage//Pepper__bell___Bact...,Pepper__bell___Bacterial_spot
1,plantdisease/PlantVillage//Pepper__bell___Bact...,Pepper__bell___Bacterial_spot
2,plantdisease/PlantVillage//Pepper__bell___Bact...,Pepper__bell___Bacterial_spot
3,plantdisease/PlantVillage//Pepper__bell___Bact...,Pepper__bell___Bacterial_spot
4,plantdisease/PlantVillage//Pepper__bell___Bact...,Pepper__bell___Bacterial_spot


In [19]:
train = data_df.sample(frac=0.8)
val = data_df.drop(train.index)
test = val.sample(frac=0.5)
val = val.drop(test.index)

print(f'Train size: {len(train)}, Validation size: {len(val)}, Test size: {len(test)}')

Train size: 16510, Validation size: 2064, Test size: 2064


In [20]:
# Create a LabelEncoder for the Labels
label_encoder = LabelEncoder()
label_encoder.fit(data_df['label'])

# Create a transform for transforming the images in the same - appropriate form
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.ConvertImageDtype(dtype=torch.float)
])

In [21]:
class PlantsDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.labels = torch.tensor(label_encoder.transform(dataframe['label']), dtype=torch.long).to(device)
        
    def __len__(self):
        return self.dataframe.shape[0]
    
    def __getitem__(self, indx):
        image = Image.open(self.dataframe.iloc[indx, 0]).convert('RGB')
        
        if self.transform:
            image = self.transform(image).to(device)
        
        label = self.labels[indx]
        
        return image, label

In [22]:
train_data = PlantsDataset(train, transform=transform)
val_data = PlantsDataset(val, transform=transform)
test_data = PlantsDataset(test, transform=transform)

print(val_data.__getitem__(1500))

(tensor([[[0.3333, 0.4431, 0.3961,  ..., 0.4824, 0.4471, 0.3922],
         [0.3294, 0.4235, 0.3882,  ..., 0.4353, 0.3608, 0.3569],
         [0.3725, 0.4235, 0.3804,  ..., 0.4118, 0.3804, 0.5176],
         ...,
         [0.6824, 0.6784, 0.7255,  ..., 0.6863, 0.6824, 0.6941],
         [0.7176, 0.6824, 0.7176,  ..., 0.7137, 0.7255, 0.7412],
         [0.7176, 0.6549, 0.6824,  ..., 0.6863, 0.6980, 0.7137]],

        [[0.2745, 0.3843, 0.3373,  ..., 0.4235, 0.3882, 0.3333],
         [0.2706, 0.3647, 0.3294,  ..., 0.3765, 0.3020, 0.2980],
         [0.3137, 0.3647, 0.3216,  ..., 0.3529, 0.3216, 0.4588],
         ...,
         [0.6392, 0.6353, 0.6824,  ..., 0.6627, 0.6588, 0.6706],
         [0.6745, 0.6392, 0.6745,  ..., 0.6902, 0.7020, 0.7176],
         [0.6745, 0.6118, 0.6392,  ..., 0.6627, 0.6745, 0.6902]],

        [[0.2863, 0.3961, 0.3490,  ..., 0.4353, 0.4000, 0.3451],
         [0.2824, 0.3765, 0.3412,  ..., 0.3882, 0.3137, 0.3098],
         [0.3255, 0.3765, 0.3333,  ..., 0.3647, 0.3333, 0

In [23]:
Learning_Rate = 1e-3
Batch_Size = 16
Epochs = 5

train_loader = DataLoader(dataset=train_data, batch_size=Batch_Size, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=Batch_Size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=Batch_Size, shuffle=True)

In [24]:
# Models
class Plants(nn.Module):
    
    def __init__(self, number_of_classes):
        super().__init__()

        # Convolutions
        self.conv2d1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2d2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv2d3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv2d4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        # Maxpooling
        self.maxpooling = nn.MaxPool2d(kernel_size=2, stride=2)

        # Activation Function
        self.activation = nn.LeakyReLU()

        # Flatten Layer
        self.flatten = nn.Flatten()

        # Dense Layers
        self.dense1 = nn.Linear((128*16*16), 256)
        self.dense2 = nn.Linear(256, 128)
        self.dense3 = nn.Linear(128, 64)
        self.output = nn.Linear(64, number_of_classes)

    def forward(self, x):
                                    # x = (3, 256, 256)
        # Stage 1
        x = self.conv2d1(x)        # (16, 256, 256)
        x = self.maxpooling(x)     # (16, 128, 128)
        x = self.activation(x)     # (16, 128, 128)
        
        # Stage 2
        x = self.conv2d2(x)        # (32, 128, 128)
        x = self.maxpooling(x)     # (32, 64, 64)
        x = self.activation(x)     # (32, 64, 64)
    
        # Stage 3
        x = self.conv2d3(x)        # (64, 64, 64)
        x = self.maxpooling(x)     # (64, 32, 32)
        x = self.activation(x)     # (64, 32, 32)

        # Stage 4
        x = self.conv2d4(x)        # (128, 32, 32)
        x = self.maxpooling(x)     # (128, 16, 16)
        x = self.activation(x)     # (128, 16, 16)

        # Stage 5
        x = self.flatten(x)
        
        # Stage 6
        x = self.dense1(x)
        x = self.activation(x)
        
        # Stage 7
        x = self.dense2(x)
        x = self.activation(x)
        
        # Stage 8
        x = self.dense3(x)
        x = self.activation(x)
        
        # Stage 9
        x = self.output(x)
        
        return x
    
class Plants_Small(nn.Module):
    
    def __init__(self, number_of_classes):
        super().__init__()

        # Convolutions
        self.conv2d1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2d2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)

        # Maxpooling
        self.maxpooling = nn.MaxPool2d(kernel_size=2, stride=2)

        # Activation Function
        self.activation = nn.LeakyReLU()

        # Flatten Layer
        self.flatten = nn.Flatten()

        # Dense Layers
        self.dense = nn.Linear((32*64*64), 32)
        self.output = nn.Linear(32, number_of_classes)

    def forward(self, x):
                                    # x = (3, 256, 256)
        # Stage 1
        x = self.conv2d1(x)        # (16, 256, 256)
        x = self.maxpooling(x)     # (16, 128, 128)
        x = self.activation(x)     # (16, 128, 128)
        
        # Stage 2
        x = self.conv2d2(x)        # (32, 128, 128)
        x = self.maxpooling(x)     # (32, 64, 64)
        x = self.activation(x)     # (32, 64, 64)

        # Stage 5
        x = self.flatten(x)
        
        # Stage 6
        x = self.dense(x)        
        x = self.output(x)
        
        return x

In [25]:
# model =  Plants(len(data_df['label'].unique())).to(device)
model =  Plants_Small(len(data_df['label'].unique())).to(device)

In [26]:
from torchsummary import summary
summary(model, (3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 256, 256]             448
         MaxPool2d-2         [-1, 16, 128, 128]               0
         LeakyReLU-3         [-1, 16, 128, 128]               0
            Conv2d-4         [-1, 32, 128, 128]           4,640
         MaxPool2d-5           [-1, 32, 64, 64]               0
         LeakyReLU-6           [-1, 32, 64, 64]               0
           Flatten-7               [-1, 131072]               0
            Linear-8                   [-1, 32]       4,194,336
            Linear-9                   [-1, 15]             495
Total params: 4,199,919
Trainable params: 4,199,919
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.75
Forward/backward pass size (MB): 19.00
Params size (MB): 16.02
Estimated Total Size (MB): 35.77
------------------------------------

In [27]:
criterion = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=Learning_Rate)

In [28]:
import time
from tqdm import tqdm

total_loss_train_plot = []
total_loss_validation_plot = []
total_acc_train_plot = []
total_acc_validation_plot = []
time_plot = []

for epoch in range(Epochs):
    epoch_start = time.time()
    total_acc_train = 0
    total_loss_train = 0
    total_loss_val = 0
    total_acc_val = 0
    
    for images, labels in tqdm(train_loader):
        optimizer.zero_grad()
        
        outputs = model(images)
        
        train_loss = criterion(outputs, labels)
        total_loss_train += train_loss.item()
        
        train_loss.backward()
        train_acc = (torch.argmax(outputs, axis=1) == labels).sum().item()
        
        total_acc_train += train_acc
        optimizer.step()
        
    with torch.no_grad():
        for images, labels in tqdm(val_loader):
            outputs = model(images)
            val_loss = criterion(outputs, labels)
            total_loss_val += val_loss.item()
            
            val_acc = (torch.argmax(outputs, axis=1) == labels).sum().item()
            total_acc_val += val_acc
            
    total_loss_train_plot.append(round(total_loss_train/1000, 4))
    total_loss_validation_plot.append(round(total_loss_val/1000, 4))
    
    total_acc_train_plot.append(round((total_acc_train/train_data.__len__()) * 100, 4))
    total_acc_validation_plot.append(round((total_acc_val/val_data.__len__()) * 100, 4))
      
    epoch_finish = round((time.time() - epoch_start)/60, 2)
        
    print(f'''Epoch {epoch+1}/{Epochs} Time: {epoch_finish} min, Train Loss: {round(total_loss_train/1000, 4)} Train Accuracy: {round((total_acc_train/train_data.__len__()) * 100, 4)}
              Validation Loss: {round(total_loss_val/1000, 4)} Validation Accuracy {round((total_acc_val/val_data.__len__()) * 100, 4)}
              ''')

100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [00:19<00:00,  6.67it/s]


Epoch 1/5 Time: 4.6 min, Train Loss: 2.5241 Train Accuracy: 21.5809
              Validation Loss: 0.2758 Validation Accuracy 34.593
              


100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [00:18<00:00,  6.94it/s]


Epoch 2/5 Time: 4.5 min, Train Loss: 1.8722 Train Accuracy: 44.2944
              Validation Loss: 0.1939 Validation Accuracy 55.1357
              


100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [00:18<00:00,  6.94it/s]


Epoch 3/5 Time: 4.51 min, Train Loss: 1.4694 Train Accuracy: 55.4634
              Validation Loss: 0.1668 Validation Accuracy 58.5756
              


100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [00:18<00:00,  6.90it/s]


Epoch 4/5 Time: 4.5 min, Train Loss: 1.2593 Train Accuracy: 61.7505
              Validation Loss: 0.1443 Validation Accuracy 66.4244
              


100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [00:18<00:00,  6.93it/s]

Epoch 5/5 Time: 4.51 min, Train Loss: 1.1235 Train Accuracy: 65.9055
              Validation Loss: 0.1314 Validation Accuracy 67.8295
              


In [29]:
# Save the trained model
torch.save(model.state_dict(), "plants_model_small.pt")

In [30]:
# Load the model
model =  Plants_Small(len(data_df['label'].unique())).to(device)
model.load_state_dict(torch.load("plants_model_small.pt"))

from torchsummary import summary
summary(model, (3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 256, 256]             448
         MaxPool2d-2         [-1, 16, 128, 128]               0
         LeakyReLU-3         [-1, 16, 128, 128]               0
            Conv2d-4         [-1, 32, 128, 128]           4,640
         MaxPool2d-5           [-1, 32, 64, 64]               0
         LeakyReLU-6           [-1, 32, 64, 64]               0
           Flatten-7               [-1, 131072]               0
            Linear-8                   [-1, 32]       4,194,336
            Linear-9                   [-1, 15]             495
Total params: 4,199,919
Trainable params: 4,199,919
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.75
Forward/backward pass size (MB): 19.00
Params size (MB): 16.02
Estimated Total Size (MB): 35.77
------------------------------------

In [31]:
# Evaluation of the trained model

from tqdm import tqdm
import time

total_loss_test = 0
total_acc_test = 0
inference_time = []

with torch.no_grad():
    for images, labels in tqdm(test_loader):
        
        start = time.time()
        outputs = model(images)
        inference_t = round((time.time() - start), 2)
        inference_time.append(inference_t)
        
        test_loss = criterion(outputs, labels)
        total_loss_test += test_loss.item()

        test_acc = (torch.argmax(outputs, axis=1) == labels).sum().item()
        total_acc_test += test_acc
        
print(f"Test Loss: {round(total_loss_test/1000, 4)}, Test Accuracy: {round(total_acc_test/test_data.__len__()*100, 4)}, Inference Time: {round(sum(inference_time)/len(inference_time), 4)} seconds")

100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [00:19<00:00,  6.60it/s]

Test Loss: 0.1308, Test Accuracy: 68.1686, Inference Time: 0.1089 seconds


In [ ]:
# Quantization and pruning of the trained model


In [ ]:
# Conversion to tflite mode of the trained model
